In [48]:
###MULTINOMIAL LOGISTIC REGRESSION
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Load the dataset
pvd = pd.read_csv("C:/Users/Jeff/Desktop/pvd.tsv", sep='\t')

columns_to_keep = [
    'collision_type', 'street_or_highway', 'nearest_intersection', 'type_of_roadway',
    'road_surface_condition', 'light_condition', 'weather_condition', 'manner_of_impact',
    'hit_and_run', 'traffic_control', 'most_serious_injury',
    'count_pedestrian', 'count_bicycle', 'scooter', 'wheel_chair'
]

pvd2 = pvd[columns_to_keep]

# Fill NA with zeros
cols = ["count_pedestrian", "count_bicycle"]
pvd2[cols] = pvd2[cols].fillna(0)

# Convert columns to factors
col_fac = ["collision_type", "street_or_highway", 'nearest_intersection', 'type_of_roadway',
           "road_surface_condition", "light_condition", "weather_condition", "manner_of_impact", "hit_and_run",
           "traffic_control", "most_serious_injury"]
pvd2[col_fac] = pvd2[col_fac].astype('category')

# Drop rows with NA values
pvd3 = pvd2.dropna()

# Calculate VIF for each feature
X_numeric = pvd3.select_dtypes(include=['number'])
vif_data = pd.DataFrame()
vif_data["feature"] = X_numeric.columns
vif_data["VIF"] = [variance_inflation_factor(X_numeric.values, i) for i in range(X_numeric.shape[1])]

# Remove columns with high VIF (e.g., VIF > 10 is often considered high)
high_vif_cols = vif_data[vif_data["VIF"] > 10]["feature"].tolist()
X_filtered = pvd3.drop(columns=high_vif_cols)

# Print the columns removed
print("Columns removed due to multicollinearity:", high_vif_cols)

# Split the data into training and test sets
X = X_filtered.drop(columns=['most_serious_injury'])
y = X_filtered['most_serious_injury']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing steps
numeric_features = ['count_pedestrian', 'count_bicycle']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())])

categorical_features = ['collision_type', 'street_or_highway', 'nearest_intersection', 'type_of_roadway',
                        'road_surface_condition', 'light_condition', 'weather_condition', 'manner_of_impact',
                        'hit_and_run', 'traffic_control']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Create a pipeline with preprocessing and logistic regression
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(multi_class='multinomial', solver='lbfgs'))])

# Fit the model
clf.fit(X_train, y_train)


# Predict on the test set
y_pred = clf.predict(X_test)
# Print classification report
print(classification_report(y_test, y_pred))



c:\Users\Jeff\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Columns removed due to multicollinearity: []
                    precision    recall  f1-score   support

 Complains Of Pain       0.68      0.95      0.79       554
             Fatal       0.00      0.00      0.00        11
    Incapacitating       0.25      0.02      0.03        62
         No Injury       0.07      0.01      0.02        85
Non-Incapacitating       0.10      0.02      0.03       104
           Unknown       0.00      0.00      0.00         3

          accuracy                           0.65       819
         macro avg       0.18      0.17      0.15       819
      weighted avg       0.50      0.65      0.54       819



c:\Users\Jeff\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Jeff\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Jeff\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labe

In [50]:
import numpy as np
from scipy import stats
from statsmodels.discrete.discrete_model import Logit

# Fit the model
clf.fit(X_train, y_train)

# Get the coefficients
coefficients = clf.named_steps['classifier'].coef_[0]
intercept = clf.named_steps['classifier'].intercept_

# Calculate the standard errors
X_train_transformed = clf.named_steps['preprocessor'].transform(X_train)
logit_model = Logit(y_train, sm.add_constant(X_train_transformed))
result = logit_model.fit(disp=0)
standard_errors = result.bse

# Calculate the z-scores
z_scores = coefficients / standard_errors

# Calculate the p-values
p_values = [2 * (1 - stats.norm.cdf(np.abs(z))) for z in z_scores]

# Print the coefficients and p-values
for i, col in enumerate(X_train.columns):
    print(f"Feature: {col}")
    print(f"Coefficient: {coefficients[i]}")
    print(f"Standard Error: {standard_errors[i]}")
    print(f"Z-score: {z_scores[i]}")
    print(f"P-value: {p_values[i]}")


c:\Users\Jeff\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AttributeError: module 'statsmodels' has no attribute 'add_constant'

In [35]:
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import RFE
import numpy as np


# Permutation Importance
perm_importance = permutation_importance(clf, X_test, y_test)
sorted_idx = perm_importance.importances_mean.argsort()

# Print permutation importance
print("Permutation Importance:")
for i in sorted_idx:
    print(f"{X.columns[i]}: {perm_importance.importances_mean[i]:.3f} +/- {perm_importance.importances_std[i]:.3f}")

Permutation Importance:
nearest_intersection: -0.009 +/- 0.003
street_or_highway: -0.007 +/- 0.005
count_pedestrian: -0.005 +/- 0.002
manner_of_impact: -0.003 +/- 0.002
road_surface_condition: -0.003 +/- 0.002
hit_and_run: -0.002 +/- 0.001
traffic_control: -0.002 +/- 0.002
light_condition: -0.001 +/- 0.002
weather_condition: -0.001 +/- 0.002
type_of_roadway: -0.000 +/- 0.003
scooter: 0.000 +/- 0.000
wheel_chair: 0.000 +/- 0.000
collision_type: 0.026 +/- 0.003
count_bicycle: 0.041 +/- 0.008


In [36]:
# Get the coefficients and intercepts from the trained logistic regression model
coefficients = clf.named_steps['classifier'].coef_
intercept = clf.named_steps['classifier'].intercept_

# Print the coefficients for each feature and class
for i, class_name in enumerate(clf.named_steps['classifier'].classes_):
    print(f"Coefficients for class '{class_name}':")
    for j, feature_name in enumerate(X_train.columns):
        print(f"  {feature_name}: {coefficients[i][j]}")
    print(f"Intercept for class '{class_name}': {intercept[i]}")
    print()


Coefficients for class 'Complains Of Pain':
  collision_type: 0.1390490093182854
  street_or_highway: 0.5246577150583364
  nearest_intersection: 0.08264535983473806
  type_of_roadway: -0.42378862047914884
  road_surface_condition: 0.7417065871791382
  light_condition: 0.207589406141592
  weather_condition: 0.30700837997867
  manner_of_impact: -0.3122001036857691
  hit_and_run: 0.19296802512899497
  traffic_control: -0.1641816749655645
  count_pedestrian: -0.4055722758025654
  count_bicycle: -0.4236078813102915
  scooter: 0.21835605667218735
  wheel_chair: 0.1612166566749147
Intercept for class 'Complains Of Pain': 1.7612340892608727

Coefficients for class 'Fatal':
  collision_type: 0.37892978603463895
  street_or_highway: -0.5687181845870778
  nearest_intersection: -0.5735405968924957
  type_of_roadway: -0.15097418150407116
  road_surface_condition: 0.20472156495836344
  light_condition: -0.000355654396076773
  weather_condition: -0.009579533290369216
  manner_of_impact: -0.0121380393

In [15]:
###RANDOM FOREST

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Load the dataset
pvd = pd.read_csv("C:/Users/Jeff/Desktop/pvd.tsv", sep='\t')

columns_to_keep = [
    'collision_type', 'street_or_highway', 'nearest_intersection', 'type_of_roadway',
    'road_surface_condition', 'light_condition', 'weather_condition', 'manner_of_impact',
    'hit_and_run', 'traffic_control', 'most_serious_injury',
    'count_pedestrian', 'count_bicycle', 'scooter', 'wheel_chair'
]

pvd2 = pvd[columns_to_keep]

# Fill NA with zeros
cols = ["count_pedestrian", "count_bicycle"]
pvd2[cols] = pvd2[cols].fillna(0)

# Convert columns to factors
col_fac = ["collision_type", "street_or_highway", 'nearest_intersection', 'type_of_roadway',
           "road_surface_condition", "light_condition", "weather_condition", "manner_of_impact", "hit_and_run",
           "traffic_control", "most_serious_injury"]
pvd2[col_fac] = pvd2[col_fac].astype('category')

# Drop rows with NA values
pvd3 = pvd2.dropna()

# Calculate VIF for each feature
X_numeric = pvd3.select_dtypes(include=['number'])
vif_data = pd.DataFrame()
vif_data["feature"] = X_numeric.columns
vif_data["VIF"] = [variance_inflation_factor(X_numeric.values, i) for i in range(X_numeric.shape[1])]

# Remove columns with high VIF (e.g., VIF > 10 is often considered high)
high_vif_cols = vif_data[vif_data["VIF"] > 10]["feature"].tolist()
X_filtered = pvd3.drop(columns=high_vif_cols)

# Print the columns removed
print("Columns removed due to multicollinearity:", high_vif_cols)

# Split the data into training and test sets
X = X_filtered.drop(columns=['most_serious_injury'])
y = X_filtered['most_serious_injury']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing steps
numeric_features = ['count_pedestrian', 'count_bicycle']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())])

categorical_features = ['collision_type', 'street_or_highway', 'nearest_intersection', 'type_of_roadway',
                        'road_surface_condition', 'light_condition', 'weather_condition', 'manner_of_impact',
                        'hit_and_run', 'traffic_control']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Create a pipeline with preprocessing and random forest classifier
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))])

# Fit the model
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)
# Print classification report
print(classification_report(y_test, y_pred))


c:\Users\Jeff\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Columns removed due to multicollinearity: []
                    precision    recall  f1-score   support

 Complains Of Pain       0.67      0.97      0.79       554
             Fatal       0.00      0.00      0.00        11
    Incapacitating       0.25      0.02      0.03        62
         No Injury       0.09      0.01      0.02        85
Non-Incapacitating       0.00      0.00      0.00       104
           Unknown       0.00      0.00      0.00         3

          accuracy                           0.66       819
         macro avg       0.17      0.17      0.14       819
      weighted avg       0.48      0.66      0.54       819



c:\Users\Jeff\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Jeff\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Jeff\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Get feature importances from the trained random forest model
feature_importances = clf.named_steps['classifier'].feature_importances_

# Create a dictionary to store feature importance scores for each response category
feature_importance_per_response = {}

# Associate features with response categories
for i, response_category in enumerate(clf.named_steps['classifier'].classes_):
    feature_importance_per_response[response_category] = feature_importances[i]

# Sort the dictionary by importance scores (optional)
sorted_feature_importance_per_response = dict(sorted(feature_importance_per_response.items(), key=lambda x: x[1], reverse=True))

# Print or visualize the results
for response_category, importance_score in sorted_feature_importance_per_response.items():
    print(f"Response Category: {response_category}, Feature Importance: {importance_score}")



Confusion Matrix:
[[537   0   3   8   6   0]
 [ 10   0   0   1   0   0]
 [ 60   0   1   1   0   0]
 [ 83   0   0   1   1   0]
 [104   0   0   0   0   0]
 [  3   0   0   0   0   0]]


AttributeError: 'LogisticRegression' object has no attribute 'feature_importances_'

In [16]:
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import RFE
import numpy as np


# Permutation Importance
perm_importance = permutation_importance(clf, X_test, y_test)
sorted_idx = perm_importance.importances_mean.argsort()

# Print permutation importance
print("Permutation Importance:")
for i in sorted_idx:
    print(f"{X.columns[i]}: {perm_importance.importances_mean[i]:.3f} +/- {perm_importance.importances_std[i]:.3f}")



Permutation Importance:
nearest_intersection: -0.013 +/- 0.002
street_or_highway: -0.009 +/- 0.002
traffic_control: -0.007 +/- 0.002
manner_of_impact: -0.005 +/- 0.002
hit_and_run: -0.005 +/- 0.003
weather_condition: -0.004 +/- 0.001
type_of_roadway: -0.004 +/- 0.001
light_condition: -0.001 +/- 0.002
count_pedestrian: -0.001 +/- 0.003
road_surface_condition: -0.000 +/- 0.002
count_bicycle: 0.000 +/- 0.004
scooter: 0.000 +/- 0.000
wheel_chair: 0.000 +/- 0.000
collision_type: 0.013 +/- 0.005
